# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Enabling widgets extensions to jupyter: (uncomment if needed)
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension
#installing gmaps (uncomment if needed)
#!pip install gmaps
# Telling jupyter to load the extension with: (uncomment if needed)
#!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output file (CSV)
output_data_file = "../output_data/cities.csv"





### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

#Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key = g_key)



In [4]:
# Create a data frame for cities

cities_df = pd.read_csv(output_data_file)
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [5]:
cities_df.dtypes

City_ID         int64
City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

In [6]:
# Data for gmaps
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

# Value for max intensity.
maxHumidity = cities_df["Humidity"].max()

# Plot a Heatmap

fig = gmaps.figure(map_type = 'HYBRID')

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,
                                 weights = humidity,
                                 dissipating = False,
                                 max_intensity = maxHumidity,# Scales based on the max humidity of the dataset.. 
                                 point_radius = 1)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down cities to fit our ideal conditions.
Hotel_df = cities_df[(cities_df['Max Temp'] >= 80) & (cities_df['Max Temp'] <= 90) & (cities_df['Cloudiness'] <= 10) & (cities_df['Wind Speed'] <= 10)]
Hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
247,247,puerto escondido,5,MX,1558378824,66,15.86,-97.07,86.00,9.17
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17
416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36
512,512,natchez,1,US,1558378892,74,31.56,-91.40,87.01,6.93
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create empty column to store hotel names
Hotel_df['Hotel Name'] = ""

# Filter the columns show only the info we want
Hotel_df = Hotel_df[['City', 'Country', 'Lat', 'Lng', 'Hotel Name']]
Hotel_df


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
20,teknaf,BD,20.86,92.31,
125,arraial do cabo,BR,-22.97,-42.02,
142,sarai mir,IN,26.02,82.92,
244,pasni,PK,25.26,63.48,
247,puerto escondido,MX,15.86,-97.07,
316,gunjur,GM,13.18,-16.76,
416,kirkuk,IQ,35.47,44.40,
512,natchez,US,31.56,-91.40,
518,bonito,BR,-21.13,-56.49,
525,mecca,SA,21.43,39.83,


In [9]:
# Set up the base url for the google nearby search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [10]:
# use iterrows to iterate through pandas dataframe
for index, row in Hotel_df.iterrows():
    # Set search. params within 5km radius
    params = {
    
    "Location": f"{row['Lat']}, {row['Lng']}",
    "radius": 50000, 
    "type": "Lodging",
    "key": g_key
    }
    
    # Response
    response = requests.get(base_url, params = params).json()
    
    # Results
    results = response['results']
    
    try:
        
        Hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        
        
        try:
            
            params = {
                "location": f"{row['Lat']}, {row['Lng']}",
                "radius": 10000,
                "type": "lodging",
                "key": g_key
            }
            response = requests.get(base_url, params = params).json()
            
            results = response['results']
            
            Hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
            
        except (KeyError, IndexError):
            
            print(f"Missing feild/result...skipping. {row['City']}")

  

Missing feild/result...skipping. teknaf
Missing feild/result...skipping. arraial do cabo
Missing feild/result...skipping. sarai mir
Missing feild/result...skipping. pasni
Missing feild/result...skipping. puerto escondido
Missing feild/result...skipping. gunjur
Missing feild/result...skipping. kirkuk
Missing feild/result...skipping. natchez
Missing feild/result...skipping. bonito
Missing feild/result...skipping. mecca
Missing feild/result...skipping. tomatlan
Missing feild/result...skipping. yabrud


In [17]:
# Removes cities without hotels in the desired radius range.
narrowed_city_df = Hotel_df[Hotel_df['Hotel Name'] != '']
    
narrowed_city_df
            

,City,Country,Lat,Lng,Hotel Name


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Get the info for each Hotel.

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = Hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                             info_box_content = hotel_info)
fig.add_layer(markers)



# Display Map
fig

IndexError: list index out of range